In [1]:
import glob

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance

from osgeo import gdal
gdal.SetConfigOption('GDAL_PDF_LAYERS', 'Map_Frame')
# gdal.SetConfigOption('GDAL_PDF_LAYERS_OFF',
#                      "Map_Frame.Projection_and_Grids")
                     
                     

In [7]:
# Coordinates of upper left corner, add -0.125N, 0.125E to get lower right
map_corners = [
    (36.5, -112.625, 'AZ_Fishtail_Mesa_20210825_TM_geo.pdf'),
    (36.5, -112.5, 'AZ_Tapeats_Amphitheater_20210825_TM_geo.pdf'),
    (36.5, -112.375, 'AZ_Timp_Point_20210821_TM_geo.pdf'),
    (36.5, -112.25, 'AZ_De_Motte_Park_20210826_TM_geo.pdf'),
    (36.5, -112.125, 'AZ_Dog_Point_20210820_TM_geo.pdf'),
    (36.5, -112.0, 'AZ_Buffalo_Ranch_20210824_TM_geo.pdf'),
    (36.5, -111.875, 'AZ_Tatahatso_Point_20210824_TM_geo.pdf'),

    (36.375, -112.625, 'AZ_Fossil_Bay_20210820_TM_geo.pdf'),
    (36.375, -112.5, 'AZ_Powell_Plateau_20210821_TM_geo.pdf'),
    (36.375, -112.375, 'AZ_King_Arthur_Castle_20210820_TM_geo.pdf'),
    (36.375, -112.25, 'AZ_Kanabownits_Spring_20210820_TM_geo.pdf'),
    (36.375, -112.125, 'AZ_Little_Park_Lake_20210820_TM_geo.pdf'),
    (36.375, -112.0, 'AZ_Point_Imperial_20210818_TM_geo.pdf'),
    (36.375, -111.875, 'AZ_Nankoweap_Mesa_20210818_TM_geo.pdf'),

    (36.25, -112.625, 'AZ_Topocoba_Hilltop_20210821_TM_geo.pdf'),
    (36.25, -112.5, 'AZ_Explorers_Monument_20210820_TM_geo.pdf'),
    (36.25, -112.375, 'AZ_Havasupai_Point_20210825_TM_geo.pdf'),
    (36.25, -112.25, 'AZ_Shiva_Temple_20210825_TM_geo.pdf'),
    (36.25, -112.125, 'AZ_Bright_Angel_Point_20210826_TM_geo.pdf'),
    (36.25, -112.0, 'AZ_Walhalla_Plateau_20210818_TM_geo.pdf'),
    (36.25, -111.875, 'AZ_Cape_Solitude_20210818_TM_geo.pdf'),

    (36.125, -112.625, 'AZ_Antelope_Point_20210820_TM_geo.pdf'),
    (36.125, -112.5, 'AZ_Chamisa_Tank_20210820_TM_geo.pdf'),
    (36.125, -112.375, 'AZ_Piute_Point_20180726_TM_geo.pdf'),
    (36.125, -112.25, 'AZ_Grand_Canyon_20210820_TM_geo.pdf'),
    (36.125, -112.125, 'AZ_Phantom_Ranch_20210821_TM_geo.pdf'),
    (36.125, -112.0, 'AZ_Cape_Royal_20210824_TM_geo.pdf'),
    (36.125, -111.875, 'AZ_Desert_View_20210824_TM_geo.pdf'),

]

In [3]:
def get_corners_from_gdal_ds(gdal_ds):
    ulx, xres, _, lry, _, yres = gdal_ds.GetGeoTransform()
    uly = orig_pdf.RasterYSize * yres + lry
    lrx = orig_pdf.RasterXSize * xres + ulx
    return ulx, uly, lrx, lry

# Get approx aspect ratio - just pick random one from the middle
fn = map_corners[len(map_corners)//2][-1]
orig_pdf = gdal.Open(fn)
ulx, uly, lrx, lry = get_corners_from_gdal_ds(orig_pdf)
ASPECT_RATIO = (ulx - lrx) / (uly - lry)

def warp_pdf(map_info):
    uly, ulx, quadrangle = map_info
    orig_pdf = gdal.Open(quadrangle)
    target_size = 4096 # equidistant
    warped = gdal.Warp(
        'warped.tif', orig_pdf, 
        dstSRS='EPSG:4326', # WGS84 
        resampleAlg='cubic',
        width=target_size, height=target_size,
        # this takes min_x, min_y, max_x, max_y
        outputBounds=(ulx, uly - 0.125, ulx + 0.125, uly)
    );
    warped.FlushCache();
    return warped

def gdal_ds_to_image(gdal_ds):
    a1 = gdal_ds.GetRasterBand(1).ReadAsArray()
    a2 = gdal_ds.GetRasterBand(2).ReadAsArray()
    a3 = gdal_ds.GetRasterBand(3).ReadAsArray()

    arr = np.stack((a1, a2, a3), axis=2)
    return Image.fromarray(arr)

def correct_aspect_ratio(im):
    return im.resize(
        (int(im.width * ASPECT_RATIO), im.height), Image.Resampling.BICUBIC
    )

def make_name_readable(fn):
    return fn.split('_20')[0][3:]
    

def do_all_the_things(map_info):
    im = correct_aspect_ratio(gdal_ds_to_image(warp_pdf(map_info)))
    im.save(f'{make_name_readable(map_info[-1])}.tif')
    return im


In [4]:
def merge_files(map_corners, crop_pixels=(0, 0, 0, 0)):
    
    row_lats = list(set(m[0] for m in map_corners))
    row_lats.sort()
    column_lons = list(set(m[1] for m in map_corners))
    column_lons.sort()

    # n.b. all input files have been transformed to the same size
    for ir, row in enumerate(row_lats):
        print(row)
        maps = [m for m in map_corners if m[0] == row]
        print(', '.join(make_name_readable(m[-1]) for m in maps))

        for ic, col in enumerate(column_lons):
            m = [m for m in maps if m[1] == col][0][-1]
            if ic == 0:
                im = Image.open(f'{make_name_readable(m)}.tif')
                im = crop_image(im, crop_pixels)
                continue 
            im_new = Image.open(f'{make_name_readable(m)}.tif')
            im_new = crop_image(im_new, crop_pixels)
            im = merge_im_horizontally(im, im_new)
        
        if ir == 0:
            im_all = im
            continue
        im_all = merge_im_vertically(im_all, im)

    return im_all

def crop_image(im, n_pixels):
    box = (n_pixels[0], n_pixels[1], im.width - n_pixels[2], im.height - n_pixels[3])
    return im.crop(box)
            
def merge_im_horizontally(im1, im2):
    w = im1.width + im2.width 
    h = im1.height
    im = Image.new("RGB", (w, h))
    im.paste(im1)
    im.paste(im2, (im1.width, 0))
    return im

def merge_im_vertically(im1, im2):
    w = im1.width
    h = im1.height + im2.height 
    im = Image.new("RGB", (w, h))
    im.paste(im2)
    im.paste(im1, (0, im2.height))
    return im

def increase_contrast(im, contrast_scale=3):
    enhancer = ImageEnhance.Contrast(im)
    return enhancer.enhance(contrast_scale)

In [10]:
for m in map_corners:
    if len(glob.glob(f'{make_name_readable(m[-1])}.tif')) == 0: # skip files that have already been processed
        print(make_name_readable(m[-1]))
        im = do_all_the_things(m)
    # break


im = merge_files(map_corners, (2, 1, 1, 2))
im.save('test.tif')
im3 = increase_contrast(im, 1.5)
sc = 1.5
im2 = im3.resize((int(im.width / sc), int(im.height / sc)), Image.Resampling.BICUBIC)
im2.save('final_map.jpg')

36.125
Antelope_Point, Chamisa_Tank, Piute_Point, Grand_Canyon, Phantom_Ranch, Cape_Royal, Desert_View
36.25
Topocoba_Hilltop, Explorers_Monument, Havasupai_Point, Shiva_Temple, Bright_Angel_Point, Walhalla_Plateau, Cape_Solitude
36.375
Fossil_Bay, Powell_Plateau, King_Arthur_Castle, Kanabownits_Spring, Little_Park_Lake, Point_Imperial, Nankoweap_Mesa
36.5
Fishtail_Mesa, Tapeats_Amphitheater, Timp_Point, De_Motte_Park, Dog_Point, Buffalo_Ranch, Tatahatso_Point
